In [8]:
import json
from prettytable import PrettyTable
import json
import os
import random
import pandas as pd

# Read the jsonl file and convert it to a JSON list
def jsonl_to_json_list(jsonl_file_path):
    json_list = []
    with open(jsonl_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            json_obj = json.loads(line.strip())  # Parse each line as JSON
            json_list.append(json_obj)
    
    return json_list

# Save the JSON list to a file
def save_as_json(json_list, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as outfile:
        json.dump(json_list, outfile, indent=4)

def save_as_jsonl(json_list, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as outfile:
        for json_obj in json_list:
            json.dump(json_obj, outfile)
            outfile.write('\n')

In [3]:
# subsample medqa 5 options
medqa_5options = jsonl_to_json_list(os.path.join('./data/medqa', 'test_5options.jsonl'))
medqa_5options = [{'realidx': i, **item} for i, item in enumerate(medqa_5options)]
random.shuffle(medqa_5options)
medqa_5options = sorted(medqa_5options[:50], key=lambda x: x['realidx'])

df = pd.DataFrame(medqa_5options)
df.head(10)

# save as jsonl
# save_as_jsonl(medqa_5options, os.path.join('./data/medqa', 'sampled_50_5options.jsonl'))


,realidx,question,answer,options,meta_info,answer_idx
0,44,A 45-year-old G5P4105 presents to her gynecolo...,Globular 10-week sized uterus,"{'A': 'Rectouterine septum nodularity', 'B': '...",step2&3,B
1,126,An 81-year-old woman is brought to the emergen...,Herniation of the uncus,"{'A': 'Herniation of the uncus', 'B': 'Herniat...",step1,A
2,146,A 60-year-old man is referred to a dermatologi...,Tumor arising from cutaneous T cells,{'A': 'Vascular tumor containing spindle cells...,step2&3,D
3,159,A 9-year-old boy is brought to a physician by ...,Bartter's syndrome,"{'A': 'Renal artery stenosis', 'B': 'Primary h...",step1,C
4,208,A clinical researcher is interested in creatin...,Inhibits viral assembly,"{'A': 'Prevents viral transcription', 'B': 'In...",step1,B
5,217,A 37-year-old primigravid woman at 12 weeks' g...,Dilation and curettage,"{'A': 'Misoprostol therapy', 'B': 'Follow-up u...",step2&3,D
6,230,An 11-year-old child complains of pain in the ...,Coarctation of the aorta,"{'A': 'Pulmonic stenosis', 'B': 'Patent ductus...",step1,C
7,243,A 61-year-old man was started on rosuvastatin...,Restart rosuvastatin at a lower dose,"{'A': 'Restart rosuvastatin at a lower dose', ...",step2&3,A
8,327,A 22-year-old man from Nepal presents to the e...,Preventable by a live attenuated vaccine,{'A': 'Preventable by a live attenuated vaccin...,step1,A
9,336,A 39-year-old man is brought to the emergency ...,Epstein-Barr virus,"{'A': 'Epstein-Barr virus', 'B': 'Human herpes...",step2&3,A


In [6]:
import json
import os
import random
import pandas as pd

# look for pubmedqa test set
pubmedqa_train_set = json.load(open(os.path.join('./data/pubmedqa', 'train_set.json'), 'r', encoding='utf-8'))
pubmedqa_train_set = [{'realidx': idx, **item} for idx, item in pubmedqa_train_set.items()]
pubmedqa_dev_set = json.load(open(os.path.join('./data/pubmedqa', 'dev_set.json'), 'r', encoding='utf-8'))
pubmedqa_dev_set = [{'realidx': idx, **item} for idx, item in pubmedqa_dev_set.items()]
pubmedqa_train_set = pubmedqa_train_set + pubmedqa_dev_set

df = pd.DataFrame(pubmedqa_train_set)

# rename columns
df_meta = df[['reasoning_required_pred', 'reasoning_free_pred', 'YEAR', 'MESHES', 'LABELS']]
df = df.drop(columns=['reasoning_required_pred', 'reasoning_free_pred', 'YEAR', 'MESHES', 'LABELS'])
df = df.rename(columns={'QUESTION': 'question', 'CONTEXTS': 'context', 'final_decision': 'answer', 'LONG_ANSWER': 'answer_text'})
df['options'] = [{'A': 'yes', 'B': 'no', 'C': 'maybe'} for _ in range(len(df))]
df['answer_idx'] = df['answer'].map({'yes': 'A', 'no': 'B', 'maybe': 'C'})
df['context'] = df['context'].apply(lambda x: '\n'.join(x))

# save df as jsonl
pubmedqa_train_set = df.to_dict(orient='records')
save_as_jsonl(pubmedqa_train_set, os.path.join('./data/pubmedqa', 'train.jsonl'))

df.head(10)

,realidx,question,context,answer,answer_text,options,answer_idx
0,18251357,Does histologic chorioamnionitis correspond to...,To evaluate the degree to which histologic cho...,yes,Histologic chorioamnionitis is a reliable indi...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
1,17606778,Can vitamin C prevent complex regional pain sy...,Complex regional pain syndrome type I is treat...,yes,Vitamin C reduces the prevalence of complex re...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
2,19504993,It's Fournier's gangrene still dangerous?,Fournier's gangrene is known to have an impact...,yes,The interval from the onset of clinical sympto...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
3,19931500,Can the condition of the cell microenvironment...,The aim of this study was to analyze the prope...,yes,The preliminary results of this study suggest ...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
4,23222920,Pancreas retransplantation: a second chance f...,If pancreas transplantation is a validated alt...,yes,Pancreas retransplantation is a safe procedure...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
5,19327500,Gender difference in survival of resected non-...,It remains controversial whether there is a ge...,yes,"Women had better survival than men; however, t...","{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
6,9792366,Is HIV/STD control in Jamaica making a differe...,To assess the impact of the comprehensive HIV/...,yes,HIV/STD control measures appear to have slowed...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
7,14976655,Delayed peripheral facial palsy in the stapes ...,The aim of this study was to evaluate poststap...,yes,Poststapedectomy-delayed facial palsy is likel...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
8,11799314,Is external palliative radiotherapy for gallbl...,Gallbladder carcinoma is characterized by dela...,yes,External radiotherapy seems to be a safe and e...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
9,17276801,Can elevated troponin I levels predict complic...,The purpose of this study was to evaluate the ...,yes,Our results indicate that elevated cTnI levels...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A


In [28]:
import json
import os
import random
import pandas as pd

# look for pubmedqa test set
pubmedqa_test_set = json.load(open(os.path.join('./data/pubmedqa', 'test_set.json'), 'r', encoding='utf-8'))
pubmedqa_test_set = [{'realidx': idx, **item} for idx, item in pubmedqa_test_set.items()]
df = pd.DataFrame(pubmedqa_test_set)
# rename columns
df_meta = df[['reasoning_required_pred', 'reasoning_free_pred', 'YEAR', 'MESHES', 'LABELS']]
df = df.drop(columns=['reasoning_required_pred', 'reasoning_free_pred', 'YEAR', 'MESHES', 'LABELS'])
df = df.rename(columns={'QUESTION': 'question', 'CONTEXTS': 'context', 'final_decision': 'answer', 'LONG_ANSWER': 'answer_text'})
df['options'] = [{'A': 'yes', 'B': 'no', 'C': 'maybe'} for _ in range(len(df))]
df['answer_idx'] = df['answer'].map({'yes': 'A', 'no': 'B', 'maybe': 'C'})
df['context'] = df['context'].apply(lambda x: '\n'.join(x))

# save df as jsonl
pubmedqa_test_set = df.to_dict(orient='records')
save_as_jsonl(pubmedqa_test_set, os.path.join('./data/pubmedqa', 'test_set.jsonl'))

sampled_50_pubmedqa = df.sample(50, random_state=42).to_dict(orient='records')
save_as_jsonl(sampled_50_pubmedqa, os.path.join('./data/pubmedqa', 'sampled_50_pubmedqa.jsonl'))

df.head(10)

,realidx,question,context,answer,answer_text,options,answer_idx
0,12377809,Is anorectal endosonography valuable in dysche...,Dyschesia can be provoked by inappropriate def...,yes,Linear anorectal endosonography demonstrated i...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
1,26163474,Is there a connection between sublingual varic...,Sublingual varices have earlier been related t...,yes,An association was found between sublingual va...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
2,19100463,Is the affinity column-mediated immunoassay me...,Tacrolimus is a potent immunosuppressive drug ...,yes,The ACMIA method used for a tacrolimus assay i...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
3,18537964,Does a physician's specialty influence the rec...,To determine the impact of a physician's speci...,yes,Physicians appear to document more frequently ...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
4,12913878,Locoregional opening of the rodent blood-brain...,Nd:YAG laser-induced thermo therapy (LITT) of ...,yes,LITT induces a locoregional passage of chemoth...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
5,12765819,Spinal subdural hematoma: a sequela of a ruptu...,A case of spinal subdural hematoma (SSDH) foll...,yes,Although the exact mechanism of SSDH in this c...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
6,25475395,Is there a correlation between androgens and s...,"For women, the correlation between circulating...",yes,"In the present study, FT and androstenedione w...","{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
7,19130332,Is the zeolite hemostatic agent beneficial in ...,Uncontrolled hemorrhage is the leading cause o...,yes,"According to the physiological parameters, we ...","{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
8,9427037,Are endothelial cell patterns of astrocytomas ...,The most common primary brain tumors in childr...,yes,Evaluation of astrocytomas utilizing antibody ...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A
9,24481006,Should cavitation in proximal surfaces be repo...,79 adjacent proximal surfaces without restorat...,yes,CBCT was more accurate in detecting cavitation...,"{'A': 'yes', 'B': 'no', 'C': 'maybe'}",A


In [11]:
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

def deduplicate_data(data):
    seen = set()
    deduplicated_data = []
    for item in data:
        idx = item['idx']
        if idx not in seen:
            deduplicated_data.append(item)
            seen.add(idx)
    return deduplicated_data

def calculate_accuracy(data):
    correct_predictions = 0
    total_predictions = len(data)
    for item in data:
        if item['label'] == item['prediction']:
            correct_predictions += 1
    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    return accuracy

tasks = {
    'medqa': ['sampled_50', 'sampled_50_hard', 'sampled_50_5options'],
    'pubmedqa': ['sampled_50']
}
models = ['gpt-4o-mini',]
difficulties = ['adaptive']

table = PrettyTable()
table.field_names = ["Model", "Task", "Subtask", "Difficulty", "Accuracy", "Total Number"]

for model in models:
    for task in tasks:
        for subtask in tasks[task]:
            for difficulty in difficulties:
                file_path = f'./output/{task}/{model}_{task}_{subtask}_{difficulty}.json'
                data = load_json(file_path)
                deduplicated_data = deduplicate_data(data)
                accuracy = calculate_accuracy(deduplicated_data)
                table.add_row([model, task, subtask, difficulty, accuracy * 100, len(deduplicated_data)])

print(table)

+-------------+----------+---------------------+------------+-------------------+--------------+
|    Model    |   Task   |       Subtask       | Difficulty |      Accuracy     | Total Number |
+-------------+----------+---------------------+------------+-------------------+--------------+
| gpt-4o-mini |  medqa   |      sampled_50     |  adaptive  |        80.0       |      50      |
| gpt-4o-mini |  medqa   |   sampled_50_hard   |  adaptive  |        36.0       |      50      |
| gpt-4o-mini |  medqa   | sampled_50_5options |  adaptive  |        68.0       |      50      |
| gpt-4o-mini | pubmedqa |      sampled_50     |  adaptive  | 51.06382978723404 |      47      |
+-------------+----------+---------------------+------------+-------------------+--------------+
